<a href="https://colab.research.google.com/github/gnishtha/Briqs/blob/master/gcpbriqmain_nish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-cloud-vision

     |████████████████████████████████| 440kB 2.8MB/s 


In [2]:
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 59 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 0s (314 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/briq-chronicle-8e5681b27d06.json"

In [4]:
import io
import os
from google.cloud import vision
from google.cloud.vision import types
import pandas as pd
import json
import cv2
import matplotlib.pyplot as plt
from collections import Counter
from dateutil import parser
import re
from google.cloud import storage
from google.protobuf import json_format
from requests.utils import requote_uri
from urllib.request import urlopen
from PIL import Image

In [5]:
from google.cloud import storage
client = storage.Client()
def getbufferofblog(bucketname,blobfile):
    bucket = client.get_bucket(bucketname)
    blob = bucket.get_blob(blobfile)
    blob.download_as_string()
    return blob.download_as_string()

In [6]:
def detect_text_v1(content):
    """Detects text in the file. Using google vision api (output will be in dataframe format)"""
    client = vision.ImageAnnotatorClient()

    # with io.open(path, 'rb') as image_file:
    #     content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    txt = []
    x0 = []
    x1 = []
    y0 = []
    y1 = []
    centerx = []
    centery = []
    for text in texts[1:]:
        #print('\n"{}"'.format(text.description))
        txt.append(text.description)
        vertices = ([(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])
        x0.append(vertices[0][0]) 
        x1.append(vertices[1][0])
        y0.append(vertices[0][1])
        y1.append(vertices[2][1])
        centerx.append(int((vertices[0][0]+vertices[1][0])/2))
        centery.append(int((vertices[0][1]+vertices[2][1])/2))
        #print('bounds: ',(vertices))
    return pd.DataFrame({"text":txt,"x0":x0,"y0":y0,"x1":x1,"y1":y1,"centerx":centerx,"centery":centery})
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [7]:
"""Take dataframe as input and add column line for each words  """
def get_sequence_data(data):
    df = data.sort_values(by = ["centery"])
    lst = list(df['centery'])
    count = 0
    line = []
    for i in range(len(lst)):
        if i != len(lst)-1:
            if lst[i+1] - lst[i] > 8:
                line.append(count)
                count += 1
            else:
                line.append(count)
        else:
            line.append(count)
    df['line'] = line
    df = df.sort_values(by=['line','x0'])
    df.reset_index(drop=True, inplace=True)
    #print("xxx")
    #print(df)
    return df

In [8]:
"""
input -> start_text of and end_text of field
output -> return info of field (for moveable data)
"""
def get_moveable_field_data(data,key,isTable):
    start_data = data[data['text'].astype(str).str.contains(key['start'], case=False)]
    end_data = data[data['text'].astype(str).str.contains(key['end'], case=False)]
    temp_dct = {}
    for ind, row in start_data.iterrows():
        temp_dct[str(ind)+"_"+str(abs(end_data['line'] - row['line']).idxmin())] = min(abs(end_data['line'] - row['line']))
    indxs = min(temp_dct, key=temp_dct.get)
    strt = int(indxs.split("_")[0])
    end = int(indxs.split("_")[1])
    end_line = data.loc[end,'line']
    strt_line = data.loc[strt,'line']
    if isTable==False:
        if end_line != strt_line:
            return data_format((" ").join(data[(data.loc[strt,'x1'] < data['x0']) & (strt_line == data['line'])]['text']),key['type'])
        else:
          return data_format((" ").join(data[(data.loc[strt,'x1'] < data['x0']) & (data.loc[end,'x0'] > data['x1'])  & (strt_line == data['line'])]['text']),key["type"])
    # else:
    #     #print(key)
    #     return get_table_structure_data(data[(end_line > data['line']) & (strt_line <= data['line'])],key,True)

In [9]:
"""
Convert the types of info field as given in yaml file
"""
def data_format(x,frmt):
    try:
        if frmt == "float":
            return float(x)
        elif frmt == "int":
            return int(x) 
        elif frmt == 'date':
            return parser.parse(x).isoformat()
        else:
            return str(x)  
    except:
        return str(x)

In [10]:
"""
return key which has particular value (used in Pipline_for_get_complete_row function)
"""
def get_key(my_dict,val): 
    for key, value in my_dict.items(): 
        if val == value: 
            return key 

In [11]:
"""
This function is use to convert fetched table as given in images 
"""
def get_complete_row(df,key):
    restructure_data = {new_list: [] for new_list in df.columns}
    for index,row in df.iterrows():
        try:
            if row[key] != "":
                for i in df.columns:
                    restructure_data[i].append(row[i])
            else:
                for i in df.columns:
                    restructure_data[i] = restructure_data[i][:-1] + [restructure_data[i][-1] + row[i]]
        except:
            pass
    return restructure_data

In [12]:
"""
input -> raw table data (unformatted)
output -> same format tabe as in image
function use-> get_key(), get_complete_row()
"""
def Pipline_for_get_complete_row(table_data):
    df = pd.DataFrame(table_data)
    tmp = {new_list: len(list(filter(lambda x: x != "", df[new_list]))) for new_list in df.columns}
    val = Counter(list(tmp.values())).most_common(1)[0][0]
    key = get_key(tmp,val)
    updated_table_data = get_complete_row(df,key)
    return updated_table_data

In [13]:
"""
This function takes google vision data, and extract the table data (unformatted)
"""
def get_table_structure_data(data,key,moveable_flag):
    if moveable_flag == True:
        df = data
    else:
        #print(data)
        index = data.loc[data[data['text'].astype(str).str.contains(key['end'], case=False) & (data['y0'] > key['y0'])].index.values[0],'line']
        df = data[(key['x0'] < data['centerx']) & (data['centerx'] < key['x1']) & (key['y0'] < data['centery']) & (data['line'] < index)]
    #print(df,"-->",data[data['text'].str.contains(key['end'], case=False) & (data['y0'] > key['y0'])],index)
    key_list = list(set(list(key.keys()))-set(['x0','x1','y0','end','start']))
    table_data = {new_list: [] for new_list in key_list}
    #print(df) 
    for i in key_list:
        #print(i)
        flag = False
        for k in list(set(list(df['line']))):
            temp_df = df[(df['line']==k) & (key[i]['x0'] < df['centerx']) & (key[i]['x1'] > df['centerx'])]
            if flag:
                #print(key[i]["type"])
                table_data[i].append((" ").join(temp_df['text']))
            flag = True
    structure_table = Pipline_for_get_complete_row(table_data)
    type_mapping = {i:key[i]['type'] for i in key.keys() if type(key[i]) == dict}
    updated_structure_table = {i:[data_format(x,type_mapping[i]) for x in structure_table[i]] for i in structure_table.keys()}
    return updated_structure_table

In [28]:
"""
input -> folder path in which images and config yaml file is stored
output-> will extract all info's of field that are mention in yaml file
"""
def get_invoice_data_v1(path,jsonfile, image):
  # if imagefiles==None:
  #   files = list(set(os.listdir(path))-set([jsonfile]))
  # else:
  #   files = [path+imagefile for imagefile in imagefiles]
    #print(files)
    
    
  data = []
  t=[]
  # for file in files:
  with open(os.path.join(path,jsonfile),'r') as f:
    mapping = json.load(f)
  #print(mapping)
  temp_df = detect_text_v1(image)
  print(temp_df)
  tmp_dict = {}
  inTableMapping={}
  for k in mapping.keys():
    print(k)
    if mapping[k]['isTable'] == 'True' and  mapping[k]['moveable'] == 'False' :
      #print(k)
      mapping[k].pop('moveable')
      mapping[k].pop('isTable')
      tmp_dict[k] = get_table_structure_data(get_sequence_data(temp_df),mapping[k],False)
    elif mapping[k]['isTable'] == 'True' and  mapping[k]['moveable'] == 'True' :
      mapping[k].pop('moveable')
      mapping[k].pop('isTable')
      tmp_dict[k] = get_table_structure_data(get_sequence_data(temp_df),mapping[k],True)
    elif mapping[k]['isTable'] == 'False' and  mapping[k]['moveable'] == 'False' :
      #print(k)
      #mapping[k].pop('moveable')
      #mapping[k].pop('isTable')
      filter_data = temp_df[(mapping[k]['x0'] < temp_df['centerx']) & (temp_df['centerx'] < mapping[k]['x1']) & (mapping[k]['y0'] < temp_df['centery']) & (temp_df['centery'] < mapping[k]['y1'])].sort_values(by='x0')
      tmp_dict[k] = data_format((" ").join(get_sequence_data(filter_data)['text']),mapping[k]["type"])
    elif mapping[k]['isTable'] == 'False' and  mapping[k]['moveable'] == 'True' :
      #print(k)
      tmp_dict[k] = get_moveable_field_data(get_sequence_data(temp_df),mapping[k],False)
  data.append(tmp_dict)
  return data
    
    

In [15]:
def getfileurllisttoprocess(bucketname,foldername, filename = ''):
    # how to get all files as list
    client = storage.Client()
    fileurllisttoprocess =[]
    input_path_detail =[]
    for blobnames in client.list_blobs(bucketname, prefix= getprefix(foldername,filename) ):
        blobfile = str(blobnames).split(', ')[1]
#         print(blobnames)
        input_path_url = r'https://storage.cloud.google.com/'+bucketname+'/' +blobfile
        input_path_url = requote_uri(input_path_url)
        input_path_detail.append([input_path_url,bucketname,blobfile])
        fileurllisttoprocess.append(input_path_detail)
    return fileurllisttoprocess

In [16]:
def getprefix(foldername,filename):
    if filename == '':
        customprefix= foldername
    else:
        customprefix= foldername + '/'+ filename
    return customprefix

In [31]:
def getImg():
  urllist = getfileurllisttoprocess('auto-invoicing-dev','test_datasciencecode')
  for i in urllist:
      inmemoryfile = getbufferofblog(i[0][1],i[1][2])
      #print(inmemoryfile)
      #image = Image.open(io.BytesIO(inmemoryfile))
      print(get_invoice_data_v1('/content/datasets/Mcmaster','config.json',inmemoryfile))

In [32]:
getImg()

               text    x0    y0    x1    y1  centerx  centery
0    MCMASTER-CARR.   104    48   663   123      383       85
1          MMASTCAR   112    51   157    56      134       53
2           CATALOG   121    72   150    79      135       75
3           Invoice  1396    82  1597   131     1496      106
4      630-600-3600   101   154   253   172      177      163
..              ...   ...   ...   ...   ...      ...      ...
132         Company   925  2104  1029  2127      977     2115
133            Page  1474  2104  1528  2126     1501     2115
134               1  1540  2105  1545  2122     1542     2113
135              of  1558  2104  1577  2121     1567     2112
136               1  1588  2104  1594  2121     1591     2112

[137 rows x 7 columns]
Invoice Number
Invoive Date
total amount
table
subtotal
tax
[{'Invoice Number': 38039914, 'Invoive Date': '2020-04-13T00:00:00', 'total amount': '', 'table': {'line': ['1 7141T41'], 'ordered': ['4Еach'], 'shiped': [4], 'Product': ['

In [33]:
urllist = getfileurllisttoprocess('auto-invoicing-dev','test_datasciencecode')
urllist

[[['https://storage.cloud.google.com/auto-invoicing-dev/test_datasciencecode/',
   'auto-invoicing-dev',
   'test_datasciencecode/'],
  ['https://storage.cloud.google.com/auto-invoicing-dev/test_datasciencecode/Invoice%2038039914%20for%20PO%204520-1.jpg',
   'auto-invoicing-dev',
   'test_datasciencecode/Invoice 38039914 for PO 4520-1.jpg'],
  ['https://storage.cloud.google.com/auto-invoicing-dev/test_datasciencecode/Invoice%2038105146%20for%20PO%20220020-1.jpg',
   'auto-invoicing-dev',
   'test_datasciencecode/Invoice 38105146 for PO 220020-1.jpg']],
 [['https://storage.cloud.google.com/auto-invoicing-dev/test_datasciencecode/',
   'auto-invoicing-dev',
   'test_datasciencecode/'],
  ['https://storage.cloud.google.com/auto-invoicing-dev/test_datasciencecode/Invoice%2038039914%20for%20PO%204520-1.jpg',
   'auto-invoicing-dev',
   'test_datasciencecode/Invoice 38039914 for PO 4520-1.jpg'],
  ['https://storage.cloud.google.com/auto-invoicing-dev/test_datasciencecode/Invoice%2038105146%2

In [19]:
!unzip '/content/drive/My Drive/datasets.zip'

Archive:  /content/drive/My Drive/datasets.zip
   creating: datasets/
   creating: datasets/flameproof/
  inflating: datasets/flameproof/config.yml  
  inflating: datasets/flameproof/Invoice (3)-1.jpg  
  inflating: datasets/flameproof/Invoice (4)-1.jpg  
  inflating: datasets/flameproof/flameproof.csv  
  inflating: datasets/flameproof/Invoice-1.jpg  
   creating: datasets/Mcmaster/
  inflating: datasets/Mcmaster/config.yml  
  inflating: datasets/Mcmaster/Invoice 38105146 for PO 220020-1.jpg  
  inflating: datasets/Mcmaster/Invoice 38039914 for PO 4520-1.jpg  
  inflating: datasets/Mcmaster/Mcmaster.csv  
   creating: datasets/excel/
  inflating: datasets/excel/config.yml  
  inflating: datasets/excel/Inv_38449_from_Excel_Disposal_LLC_8496-1.jpg  
  inflating: datasets/excel/Inv_39439_from_Excel_Disposal_LLC_8680-1.jpg  
  inflating: datasets/excel/excel.csv  
   creating: datasets/walnut/
  inflating: datasets/walnut/config.yml  
  inflating: datasets/walnut/IN125ADJ-1.jpg  
  infla